In [1]:
import os
import random
from PIL import Image
import argparse
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from fastai.vision.models import xresnet

In [2]:
team_id = 15
team_name = "loSSLess"
email_address = "vvb238@nyu.edu"

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, split, transform):
        r"""
        Args:
            root: Location of the dataset folder, usually it is /dataset
            split: The split you want to used, it should be one of train, val or unlabeled.
            transform: the transform you want to applied to the images.
        """

        self.split = split
        self.transform = transform

        self.image_dir = os.path.join(root, split)
        label_path = os.path.join(root, f"{split}_label_tensor.pt")

        self.num_images = len(os.listdir(self.image_dir))

        if os.path.exists(label_path):
            self.labels = torch.load(label_path)
        else:
            self.labels = -1 * torch.ones(self.num_images, dtype=torch.long)

    def __len__(self):
        return self.num_images

    def __getitem__(self, idx):
        with open(os.path.join(self.image_dir, f"{idx}.png"), 'rb') as f:
            img = Image.open(f).convert('RGB')

        return self.transform(img), self.labels[idx]

In [4]:
train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
trainset = CustomDataset(root='/dataset', split="train", transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

In [6]:
import torchvision

In [7]:
class SwaVRes34(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoder=torch.nn.Sequential(*(list(xresnet.xresnet18(pretrained=False).children()))[:-1])
        checkpoint = torch.load(f"./encoder_xres_v2.pth",map_location=lambda storage, loc: storage)
        self.encoder.load_state_dict(checkpoint)
        self.classifier=nn.Linear(in_features=512,out_features=800)
    
    def forward(self,x):
        rep=self.encoder(x).view(x.shape[0],-1)
        y_hat=self.classifier(rep)
        return y_hat

In [8]:
model=SwaVRes34()

In [9]:
net = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

In [10]:
print('Start Training')
numOfBatches = len(trainset)/trainloader.batch_size

for epoch in range(50):
    net.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader), total=int(numOfBatches)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        outputs = net(inputs)
#         break
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if i % 10 == 9:    # print every 10 mini-batches
#             print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
#             running_loss = 0.0
#     break
    print("Loss at epoch", epoch, "is", running_loss/numOfBatches)
    scheduler.step(running_loss)

print('Finished Training')

Start Training


100%|██████████| 400/400 [00:39<00:00, 10.05it/s]

Loss at epoch 0 is 6.6930344200134275



100%|██████████| 400/400 [00:26<00:00, 15.07it/s]

Loss at epoch 1 is 6.232588925361633


...

Loss at epoch 47 is 3.4601959371566773



100%|██████████| 400/400 [00:28<00:00, 14.13it/s]

Loss at epoch 48 is 3.4572765928506852



100%|██████████| 400/400 [00:28<00:00, 14.26it/s]

Loss at epoch 49 is 3.430015920996666
Finished Training


In [11]:
save_model_location = './'
os.makedirs(save_model_location, exist_ok=True)
torch.save(net.state_dict(), os.path.join(save_model_location, "xx_resnet18_v2.pth"))

In [12]:
evalset = CustomDataset(root='/dataset', split="val", transform=eval_transform)
evalloader = torch.utils.data.DataLoader(evalset, batch_size=256, shuffle=False, num_workers=2)

In [13]:
# net = model.cuda()

net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in evalloader:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f"Team {team_id}: {team_name} Accuracy: {(100 * correct / total):.2f}%")

Team 15: loSSLess Accuracy: 20.90%
